# Обучение с использованием одного нейрона в Flux.jl

В этой записной книжке мы будем использовать `Flux` для создания одного нейрона и обучим его, как мы это делали вручную в записной книжке 10!

### Чтение данных и их обработка

Давайте начнем с чтения в наших данных

In [ ]:
using CSV
using TextParse
using DataFrames

apples = CSV.read("data/Apple_Golden_1.csv")
bananas = CSV.read("data/bananas.csv")

и обрабатаем их для извлечения информации о красной и зеленой окраске на наших изображениях:

In [ ]:
col1 = :red
col2 = :green

x_apples  = [ [apples[i, col1], apples[i, col2]] for i in 1:size(apples)[1] ]
x_bananas = [ [bananas[i, col1], bananas[i, col2]] for i in 1:size(bananas)[1] ]

xs = vcat(x_apples, x_bananas)

ys = vcat( zeros(size(x_apples)[1]), ones(size(x_bananas)[1]) );

Входные данные находятся в `xs`, а метки (истинные классификации как бананы или яблоки) - в` ys`.

### Использование `Flux.jl`

Теперь мы можем загрузить `Flux`, чтобы начать работу!

In [ ]:
using Flux

В последнем блокноте мы видели, что σ является встроенной функцией в Flux. Другая функция, которая часто используется в нейронных сетях, называется `ReLU`; в Julia эта функция вызывается `relu`.

#### Упражнение 1

Используйте документы, чтобы узнать, что такое ReLU.

`relu.([-3, 3])` вернет

A) [-3, 3] <br>
B) [0, 3] <br>
C) [0, 0] <br>
D) [3, 3] <br>

### Создание одного нейрона в Flux

Давайте используем `Flux` для построения нашего нейрона с 2 входами и 1 выходом:

 <img src="data/single-neuron.png" alt="Drawing" style="width: 500px;"/>
 
 Ранее мы поместили два веса в вектор, $ \mathbf {w} $. Вместо этого Flux помещает веса в матрицу $ 1 \times 2 $ (то есть матрицу с 1 *строкой* и 2 *столбцами*). 
 
 Ранее, чтобы вычислить скалярное произведение $ \mathbf {w} $ и $ \mathbf {x} $, мы должны были использовать либо функцию `dot`, либо мы должны были транспонировать вектор $ \mathbf {w} $. :
 
```julia
# transpose w
b = w' * x
# or use dot!
b = dot(w, x)
```
Если вместо этого веса сохраняются в матрице $ 1 \times 2 $, `W`, то вместо этого мы можем просто умножить` W` и `x`! Мы начинаем со случайных значений наших параметров и данных:

In [ ]:
W = rand(1, 2)

In [ ]:
x = rand(2)

Обратите внимание, что произведение `W` и` x` теперь будет массивом (вектором) с одним элементом, а не одним числом:

In [ ]:
W * x

Это означает, что наше смещение `b` обрабатывается как массив, когда мы используем` Flux`:

In [ ]:
b = rand(1)

#### Упражнение 2

Напишите функцию `mypredict`, которая будет принимать один вход, массив` x` и использовать `W`,` b` и встроенный `σ` для генерации выходного прогноза (сохраненного в виде массива). Эта функция определяет нашу нейронную сеть! 

Подсказка: эта функция будет очень похожа на $ f _ {\mathbf {w}, \mathbf {b}} $ из последней записной книжки, но изменилась с тех пор, как изменились наши структуры данных для хранения наших параметров!

#### Упражнение 3

Определите функцию потерь, называемую `loss`. Она должна принимать два входа: вектор ` x`, в котором хранятся данные  и вектор, в котором хранятся правильные 'метки' для этих данных. `loss` должен возвращать сумму квадратов различий между предсказаниями и правильными метками.

## Расчет градиентов с помощью Flux: обратное распространение ошибки

Для обучения мы знаем, что нам нужен способ вычисления производных функции `loss` по параметрам «W» и «b». До сих пор мы делали это, используя конечные разности. 

Вместо этого `Flux.jl` реализует числовой метод, называемый **backpropagation**, который вычисляет градиенты (по существу) точно, автоматически, путем косвенного применения правил исчисления. 

Для этого он предоставляет новый тип объектов, называемый «отслеживаемыми» массивами. Это массивы, которые хранят не только их текущее значение, но также информацию о градиентах, которая используется методом обратного распространения. Если вы хотите понять математику обратного распространения, мы рекомендуем, например, [эту лекция](https://www.youtube.com/watch?v=i94OvYb6noo) [почитать на русском](https://habr.com/ru/post/348028/).

Для этого `Flux` предоставляет функцию` param` для определения таких объектов, которые будут содержать информацию о параметрах.

Начнем, как обычно, с установки некоторых случайных начальных значений для параметров:

In [ ]:
W_data = rand(1, 2)  
b_data = rand(1)

W_data, b_data

Теперь мы настроим объекты `Flux.jl`, которые будут содержать эти значения *и* их производные:

In [ ]:
W = param(W_data)
b = param(b_data)

Здесь `param` - это функция, которую предоставляет Flux для создания объекта, который представляет параметр модели машинного обучения, то есть объект, который имеет как значение, так и информацию о производной.

#### Упражнение 4

Какой тип у `W`?

A) Array (1D) <br>
B) Array (2D) <br>
C) TrackedArray (1D) <br>
D) TrackedArray (2D) <br>
E) Parameter (1D) <br>
F) Parameter (2D) <br>

#### Упражнение 5

`W` хранит не только его текущее значение, но также имеет место для хранения информации о градиенте. 

Вы можете получить доступ к значениям и градиенту весов следующим образом: 
```julia 
W.data 
W.grad 
``` 
Что больше на этом этапе?  
A) значения весов  
B) градиент весов

#### Упражнение 6

Для данных `x` и `y` где

```julia
x, y = [0.413759, 0.692204], [0.845677]
```
примените функцию потерь к `x` и` y`, чтобы получить новую переменную `l`. Какой тип `l`? (Сколько у нее измерений?)

A) Array (0D) <br>
B) Array (1D) <br>
C) TrackedArray (0D) <br>
D) TrackedArray (1D)<br> 
E) Float64<br>
F) Int64<br>

### Стохастический градиентный спуск

Теперь мы можем использовать эти функции для переопределения стохастического градиентного спуска, следуя методу, который мы использовали в предыдущем блокноте, но теперь используя обратное распространение!

#### Упражнение 7

Измените код из предыдущей записной книжки для стохастического градиентного спуска, чтобы вместо него использовать Flux.

### Исследование стохастического градиентного спуска

Давайте посмотрим на значения, хранящиеся в `b`, перед тем как запустить стохастический градиентный спуск:

In [ ]:
b

После запуска `stochastic_gradient_descent` мы находим следующее:

In [ ]:
W_final, b_final = stochastic_gradient_descent(loss, W, b, xs, ys, 100000)

мы можем взглянуть на значения `W_final` и` b_final`, которые наша машина научила генерировать желаемую классификацию.

In [ ]:
W_final

In [ ]:
b_final

#### Упражнение 8

Отобразите на графике данные и обученную функцию

#### Упражнение 9

Отображайте промежуточные этабы обучения, чтоб получить анимацию

### Автоматизация с Flux.jl

Нам нужно будет повторить вышеописанный процесс для множества различных систем. К счастью, `Flux.jl` предоставляет нам инструменты для автоматизации этого!

Flux позволяет создать нейрон простым способом:

In [ ]:
using Flux

In [ ]:
model = Dense(2, 1, σ)

`2` и` 1` относятся к числу входов и выходов, а нейрон определяется с помощью функции $ \sigma $.

In [ ]:
typeof(model)

Мы создали объект типа «Dense», определенный как «Flux», с именем «model». Это так называемый «плотный слой нейронной сети» (или полносвязный перцептрон). Параметры, которые будут изменены в процессе обучения, живут *внутри* объекта `модель`.

#### Упражнение 10

Выясните, какие переменные живут внутри объекта `model` и какого они типа. Как это соотносится с вызовом для создания «плотного» объекта, с которого мы начали?

### Модельный объект как функция

Мы можем применить объект `model` к данным, как если бы это была стандартная функция:

In [ ]:
model(rand(2))

#### Упражнение 11

Докажите себе, что вы понимаете, что происходит, когда мы называем модель. Создайте два массива `W` и` b` с теми же элементами, что и у `model.W` и` model.b`. Используйте `W` и` b` для генерации того же ответа, который вы получаете, когда мы называем `model ([. 5, .5])`.

### Использование Flux

Теперь нам нужно предоставить Flux три элемента информации: 
1. Функция потерь 
2. Некоторые обучающие данные 
3. Метод оптимизации

### Функции потери

В Flux встроены различные функции потерь, например, среднеквадратическая ошибка (`mse`), которую мы уже использовали:

In [ ]:
loss(x, y) = Flux.mse(model(x), y)

Еще часто используется перекрестная энтропия, `Flux.crossentropy`.

### Данные

Данные могут принимать несколько различных форм. Одна из них - это **итератор**, состоящий из пар $ (x, y) $ данных и меток. Мы можем добиться этого с помощью zip.

#### Упражнение 12

Используйте `zip`, чтобы' сжать '` xs` и `ys`. Затем используйте функцию `collect`, чтобы проверить, что на самом деле делает` zip`.

### Методы оптимизации

Теперь нам нужно сообщить Flux, какую процедуру оптимизации использовать. Он имеет несколько встроенных; используемый нами алгоритм стохастического градиентного спуска называется SGD. Мы должны передать ему две вещи: список объектов параметров, которые будут изменены подпрограммой оптимизации, и размер шага:

In [ ]:
opt = SGD([model.W, model.b], 0.01)
# дать список параметров, которые будут изменены

Вычисления градиента и обновления параметров будут выполняться этой функцией оптимизатора; мы не видим этих подробностей, но если вам интересно, вы можете, конечно, взглянуть на исходный код `Flux.jl`!

### Обучение

Теперь у нас есть все, чтобы на самом деле **обучить** нашу модель (один нейрон) на данных. «Обучение» относится к использованию предварительно помеченных данных для изучения функции, которая связывает входные данные с желаемыми выходными данными, указанными метками. 

`Flux` предоставляет функцию` train! `, которая выполняет один проход через данные и выполняет один шаг оптимизации, используя функцию частичной стоимости для каждой точки данных:

In [ ]:
Flux.train!(loss, data, opt)

Затем мы можем просто повторить это несколько раз, чтобы обучить сеть еще больше и настроить ее к минимуму функции стоимости:

In [ ]:
for i in 1:100
    Flux.train!(loss, data, opt)
end

Теперь давайте посмотрим на параметры после тренировки:

In [ ]:
model.W

In [ ]:
model.b

Вместо того, чтобы записывать список параметров для изменения, `Flux` предоставляет функцию` params`, которая извлекает все доступные параметры из модели:

In [ ]:
opt = SGD(params(model), 0.01)

In [ ]:
params(model)

## Добавление дополнительных функций

#### Упражнение 13

До сих пор мы только что использовали два признака, красный и зеленый цвета. 

1. Добавьте третью функцию, синюю. Подготовьте новые данные. 
1. Обучить нейрон с 3 входами и 1 выходом по данным. 
1. Можете ли вы найти хороший способ визуализировать результат?